## Random Forest with 3-fold CV, StandardScaler and optuna on dataset3.0
dataset: 11 features,  subsampled dataset

20220926

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import optuna as opt
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [2]:
#import data
train_df = pd.read_parquet("train_main.parquet.snappy")
test_df = pd.read_parquet("test_main.parquet.snappy")
test_df

,r_id,r_stars,r_stars_square,r_length,u_friends_count,u_review_count,u_month_age,b_stars,b_review_count,r_sen,r_sub,r_rea,r_useful
0,4195320,4,16,86,163,40,27.298941,4.0,445,0.117031,0.361875,86.10,1
1,4406379,5,25,28,18,115,0.001011,4.0,636,-0.140000,0.133333,96.48,3
2,1172072,5,25,76,69,1,0.000002,2.5,15,0.256723,0.355398,69.07,1
3,1949778,1,1,186,23,6,23.288288,3.5,185,-0.209252,0.378994,82.54,1
4,4377517,5,25,57,1,25,48.728723,4.5,227,0.295040,0.454762,91.11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5373252,5,25,25,288,2,24.026242,3.5,117,0.775000,0.662500,64.04,1
99996,4500278,5,25,76,21,59,44.444338,4.0,532,0.325664,0.561008,71.48,1
99997,4981139,5,25,65,236,143,4.868817,4.0,7568,0.097821,0.502692,83.66,2
99998,3032513,1,1,33,2,28,0.002720,3.0,80,0.300000,0.306250,100.04,3


In [3]:
X_train = train_df.drop(['r_id','r_useful'],axis=1)
y_train = train_df['r_useful']
X_test = test_df.drop(['r_id','r_useful'],axis=1)
y_test = test_df['r_useful']

In [4]:
def objective(trial):
    start = time.time()
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 1, 500, 50),
        "max_depth": trial.suggest_int('max_depth', 1, 500, 50),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 20),
        "max_features" : trial.suggest_categorical("max_features", ['sqrt', 'log2']),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20), 
        "random_state": 760, 
        "n_jobs": -1
    }
    print("Currently running with:")
    print(params)
    
    model = RandomForestRegressor(**params)
    pipe = make_pipeline(StandardScaler(), model)
    
    # model.fit(X_train, y_train)
    cvscore = np.sqrt(-np.mean(cross_val_score(pipe, X_train, y_train, scoring = "neg_mean_squared_error", cv=3)))
    # y_pred = model.predict(X_val)
    print("Time cost:", time.time()-start)
    # return mean_squared_error(y_val, y_pred, squared=False)
    return cvscore

In [5]:
study = opt.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 50)
study.best_params

[I 2022-09-30 19:15:41,413] A new study created in memory with name: no-name-b5677a4c-2564-4612-9bd3-ca450afe6287


Currently running with:
{'n_estimators': 301, 'max_depth': 51, 'min_samples_split': 16, 'max_features': 'sqrt', 'min_samples_leaf': 13, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-30 19:17:01,620] Trial 0 finished with value: 3.607568401544493 and parameters: {'n_estimators': 301, 'max_depth': 51, 'min_samples_split': 16, 'max_features': 'sqrt', 'min_samples_leaf': 13}. Best is trial 0 with value: 3.607568401544493.


Time cost: 80.20488619804382
Currently running with:
{'n_estimators': 201, 'max_depth': 451, 'min_samples_split': 2, 'max_features': 'sqrt', 'min_samples_leaf': 13, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-30 19:17:58,940] Trial 1 finished with value: 3.6090340879103766 and parameters: {'n_estimators': 201, 'max_depth': 451, 'min_samples_split': 2, 'max_features': 'sqrt', 'min_samples_leaf': 13}. Best is trial 0 with value: 3.607568401544493.


...

[I 2022-09-30 20:27:39,126] Trial 48 finished with value: 3.5726656847514624 and parameters: {'n_estimators': 301, 'max_depth': 401, 'min_samples_split': 13, 'max_features': 'log2', 'min_samples_leaf': 5}. Best is trial 13 with value: 3.5417859570521797.


Time cost: 93.5672357082367
Currently running with:
{'n_estimators': 351, 'max_depth': 401, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 2, 'random_state': 760, 'n_jobs': -1}


[I 2022-09-30 20:29:41,894] Trial 49 finished with value: 3.5507249860295413 and parameters: {'n_estimators': 351, 'max_depth': 401, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 2}. Best is trial 13 with value: 3.5417859570521797.


Time cost: 122.76592135429382


{'n_estimators': 351,
 'max_depth': 251,
 'min_samples_split': 2,
 'max_features': 'log2',
 'min_samples_leaf': 1}

In [8]:
#optimal model
# {'n_estimators': 351, 'max_depth': 251, 'min_samples_split': 2, 'max_features': 'log2', 'min_samples_leaf': 1}
omodel = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators = 351, 
                               max_depth = 251, 
                               min_samples_leaf= 1,
                               min_samples_split= 2,
                               max_features= 'log2', random_state = 760, n_jobs=-1))


omodel.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=251, max_features='log2',
                                       n_estimators=351, n_jobs=-1,
                                       random_state=760))])

In [9]:
#predict train set results
y_pred = omodel.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
mae = mean_absolute_error(y_train, y_pred) 
print(f"\n model train score -> RMSE:", rmse, "MAE:", mae)


#predict test set results
y_pred_test = omodel.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_test, squared=False)
mae = mean_absolute_error(y_test, y_pred_test) 
print(f"\n model test score -> RMSE:", rmse, "MAE:", mae)


 model train score -> RMSE: 1.2902994094058773 MAE: 0.5922162862201784

 model test score -> RMSE: 4.086867282916606 MAE: 1.6254888126088403
